An Antipattern is a common solution to a problem that over-all makes things worse than they could have been.
This blog post aims to highlight a few antipatterns common in Julia code.
I suspect a lot of this is due to baggage brought from other languages, where these are not Antipatterns, but are infact good pattterns.
This post is to clear things up.
<!--more-->

This post will not cover anything in the [Julia Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/).

Each section in this will decribe a different antipattern.
It will discuss:
 - Examples, including highlighting this issues
 - What to do instead
 - Reasons people do this


## NotImplementedErrors

This shows up in packages defining APIs that others should implement.
Where one wants to declare and document functions that implementors of the API should overload.
In this antipattern, one declares an abstract type, and a function that takes that abstract type as an argument.
Usually, as the the first abstract following something like fashion from Python etc, as taking the object as the first argument.
This function would throw an error saying that the function was not implemented for this type.

The logic being if someone only implements half the API, the user would get this "Helpful" error message.

In [5]:
abstract type AbstractModel end

"""
    probability_estimate(model, observation::AbstractArray)::Real

For a given `model`, returns the likelyhood of the `observation` occuring.
"""
function probability_estimate(model::AbstractModel, observation::AbstractArray)
    error("`probability_estimate` has not been implemented for $(typeof(model))")
end


## Now a type implementing this API:
"""
    GuessingModel <: AbstractModel

A model that just guesses. Not even educated guesses. Just random guessing.
"""
struct GuessingModel <: AbstractModel
end

probability_estimate(guesser::GuessingModel, observation::AbstractMatrix) = rand()

probability_estimate (generic function with 2 methods)

In [2]:
probability_estimate(GuessingModel(), [1,2,3])

ErrorException: `probability_estimate` has not been implemented for GuessingModel

**What happened?**  
It sure looks like `GuessingModel` has implemented a `probability_estimate` method.
The error does not at all help us see what is wrong.
Astute eyed readers will see what is wrong:
`GuessingModel()` was incorrectly implemented to only work for `AbstractMatrix`, but it was called with a `Vector`,
 so it fell back to the generic method for `AbstractModel`.
But that error message what not informative, and if we hit that deep inside a program we would have no idea what is going on, because it doesn't print the types of all the arguments.

### What to do instead?
Just don't implement the thing you don't want to implement.
A `MethodError` indicates this quite well and as shown gives a more informative error message than you will write.

A often missed feature is that you can declare a function without providing any methods.
This is the ideal way to add documentation for a function that you expect to be overloaded.
This is done via `function probability_estimate end`.
As shown. (using `probability_estimate2` to show how it should be done correctly)

In [9]:
"""
    probability_estimate2(model, observation::Vector)::Real

For a given `model`, returns the likelyhood of the `observation` occuring.
"""
function probability_estimate2 end

probability_estimate2(guesser::GuessingModel, observation::AbstractMatrix) = rand()

probability_estimate2 (generic function with 1 method)

In [10]:
probability_estimate2(GuessingModel(), [1,2,3])

MethodError: MethodError: no method matching probability_estimate2(::GuessingModel, ::Array{Int64,1})
Closest candidates are:
  probability_estimate2(::GuessingModel, !Matched::Array{T,2} where T) at In[9]:8

### Aside: Test Suites, for interface testing.
Not the topic of this blog post, but as an aside:
When one is in this situtation, defining a interface that another package would implement, one can provide a test-suite for testing it was implemented correctly.
This is a function they can call in their tests to at least check they have the basics right.
This can take the place of a formal interface (which julia doesn't have), in ensuring that a contract is being met.

In [11]:
using Test
function model_testsuite(model)
    @testset "Model API Test Suite for $(typeof(model))" begin
        #...
        @testset "probability_estimate" begin
            p = probability_estimate(model, [1,2,3])
            @test p isa Real
            @test 0 <= p <= 1            
        end
        #...
    end
end

model_testsuite(GuessingModel())

probability_estimate: Error During Test at In[11]:5
  Got exception outside of a @test
  `probability_estimate` has not been implemented for GuessingModel
  Stacktrace:
   [1] error(::String) at ./error.jl:33
   [2] probability_estimate(::GuessingModel, ::Array{Int64,1}) at ./In[5]:4
   [3] macro expansion at ./In[11]:6 [inlined]
   [4] macro expansion at /usr/local/src/julia/julia-1.4/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113 [inlined]
   [5] macro expansion at ./In[11]:6 [inlined]
   [6] macro expansion at /usr/local/src/julia/julia-1.4/usr/share/julia/stdlib/v1.4/Test/src/Test.jl:1113 [inlined]
   [7] model_testsuite(::GuessingModel) at ./In[11]:5
   [8] top-level scope at In[11]:13
   [9] eval at ./boot.jl:331 [inlined]
   [10] softscope_include_string(::Module, ::String, ::String) at /Users/oxinabox/.julia/packages/SoftGlobalScope/cSbw5/src/SoftGlobalScope.jl:218
   [11] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/oxinabox/.julia/packages/IJulia/yLI42/src/execute

TestSetException: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.

## Over constraining argument types

Type constraints in Julia are **only** for dispatch.
If you don't have multiple methods for a function, you don't need any type-constraints.
If you must add type-constraints (for dispatch) do so as losely as possible.


### Reasons people do this:
I think this comes from three main places.
The belief it would make the code faster (false), safer (mostly false), or easier to understand (true)
The first two come from different languages which do not act like Julia.
The last point on ease of understanding is absolutely true, but not worth it most of the time.
One other reason I can imagine is misunderstanding [this part of the documentation](https://docs.julialang.org/en/v1/manual/performance-tips/), which applies to `struct` fields, not arguments.
I hope that misunderstanding is not a common reason.

The belief that adding type constraints makes code faster, comes from not understanding how the JIT compiler works.
Julia specializes every function on the types of all arguments.
This means it generates different machine code that is more optimally suited to the particular types.
This includes things like removing branches that can't be met by this type, static dispatches, as well as actually better CPU instructions than a normal dynmaic language might use.
One can see this change by comparing `@code_typed ((x)->2x)(1.0)` vs `@code_typed ((x)->2x)(0x1)`.
Some languages, for example [Cython](https://cython.readthedocs.io/en/latest/src/quickstart/cythonize.html#faster-code-via-static-typing), *do* become much faster with type-annotations, as they do not have a JIT specializing every function when it occurs.
They do their code generation ahead of time so either have to handle all cases (if not specified) or can optimize for a particular (if specified).
In Julia the code generated for a function will be just as fast with or without type constraints.

The belief that adding type constraints makes code safer, comes from the idea of [type-safety](https://en.wikipedia.org/wiki/Type_safety).
A great advantages of statically-typed ahead-of-time compiled languages is the ability at compile time to catch and report programmer errors using the type system and looking for violated constraint.
Julia is not one of these languages, it is not statically typed so reasoning about types can only ever be partial, and Julia is not ahead to time compiled, so any errors could not be reported til the code is executing anyway.
Julia also don't have the formal notion of an interface or contract assert in the first place.
This lack does have a nice advantage in how duck-typing can allow for simpler compositionality -- by assuming it works and implementing only the parts that don't.
See my earlier [post on this](https://white.ucc.asn.au/2020/02/09/whycompositionaljulia.html#multiple-dispatch--duck-typing).
_Errors will be thrown eventually, when you do something unsupported.

The reason one I do think holds some water, is for understandability.
Putting in type-constraints on function arguments makes them easier to understand.
This is true, it is much clear what: `apply_inner(f::Function, c::Vector{<:Vector})` does, vs `apply_inner(f, c)` does.
But we have other tools to make this clear.
For a start better names, e.g. `apply_inner(func, list_of_lists)`.
As well as documentation: we can and should, put a docstring on most functions.
But I do concede sometimes on this, especially when following the norms of a particular code-base.
I will on occation add a type-constraint because I feel like it does clarify things a lot though.

### Examples

There are many examples of this; and the problems it causes.

### Requiring `AbstractVector` when one just needs an iterator

I think this one is common when people feel uncomfortable that nothing has asserted that their input was iterable, since Julia does not have a type to represent being iterable.
But thats not actually a problem as the assertion will occur when you try to iterate it -- no need to pre-empt it.

In [6]:
function my_average(xs::AbstractVector)
    len=0
    total = zero(eltype(xs))
    for x in xs
        len+=1
        total += x
    end
    return total/len
end

my_average (generic function with 1 method)

What goes wrong? There are useful iterators that do not subtype `AbstractVector`.
And converting them into `AbstractVector` via `collect(itr)` would allocate unnecessary memory, which we strive to avoid in high performance code.

In [13]:
my_average((1, 3, 4))

MethodError: MethodError: no method matching my_average(::Tuple{Int64,Int64,Int64})
Closest candidates are:
  my_average(!Matched::AbstractArray{T,1} where T) at In[6]:2

In [9]:
data = [1, 2, 3, missing, 5, 4, 3, 2, 1]
my_average(skipmissing(data))

MethodError: MethodError: no method matching my_average(::Base.SkipMissing{Array{Union{Missing, Int64},1}})
Closest candidates are:
  my_average(!Matched::AbstractArray{T,1} where T) at In[6]:2

### Dispatching on  `AbstractVector{<:Real}` rather than `AbstractVector`

*\*old man here, sitting in a rocking chair\**
*Back in my day we didn't have none of this fancy triangular dispatch, and we were just fine.*

But seriously, the ability to dispatch on the the fact that your type parameter was some subtype of an abstract type, was not introduced into the language until Julia 0.6; and before then people got on just fine.
You actually need this very rarely, because if types are used only for dispatch, you must have both
`AbstractVector{<:Real}` and some other alternative like `AbstractVector{<:AbstractString}` or plain `AbstractVector` also being dispatched on.
And its is generally pretty weird to have the need for a different implementation depending on the element type.
(It happens, but you basically need to be implementing performance optimizations)

In [36]:
using BenchmarkTools
terrible_norm(x::AbstractVector{<:Real}) = only(reshape(x, 1, :) * x)

terrible_norm (generic function with 1 method)

In [39]:
terrible_norm(1:10)

385

The thing that can go wrong here is that there are many kinds of element types that might only hold real numbers, but that do not have that as a type parameter.

For example, the if the data ever contained `missing` values (common in data science),
but you filtered them out somehow, the array will still be typed `Union{Missing, T}`

In [45]:
data = [1, 2, 3, missing]
terrible_norm(@view(data[1:3]))

MethodError: MethodError: no method matching terrible_norm(::SubArray{Union{Missing, Int64},1,Array{Union{Missing, Int64},1},Tuple{UnitRange{Int64}},true})
Closest candidates are:
  terrible_norm(!Matched::AbstractArray{#s6,1} where #s6<:Real) at In[36]:2

Or from source that *could* contrain non-Real values, but that actually doesn't

let
    x = []
    for ii in 1:10
        push!(x, ii)
    end
    
    terrible_norm(x)
end

### Dispatching on  `Function`


In [74]:
apply_inner(func::Function, xss) = [[func(x) for x in xs] for xs in xss]

apply_inner (generic function with 1 method)

In [77]:
apply_inner(round, [[0.2, 0.9], [1.2, 1.3, 1.6]])

2-element Array{Array{Float64,1},1}:
 [0.0, 1.0]
 [1.0, 1.0, 2.0]

But this doesn't work on callable objects that don't subtype `Function`.
Which include constructors.

In [82]:
apply_inner(Float32, [[0.2, 0.9], [1.2, 1.3, 1.6]])

MethodError: MethodError: no method matching apply_inner(::Type{Float32}, ::Array{Array{Float64,1},1})
Closest candidates are:
  apply_inner(!Matched::Function, ::Any) at In[74]:1

A workaround to that is to use `Base.Callable` which is a `Union{Type, Function}` so does functions and constructors.
However, this will still miss-out on other callable objects, like `DiffEqBase.ODESolution` and `Flux.Chain`.
(As `Base.Callable` is not exported, it is also probably not part of the intended public API of Julia.)

### Others
There are a few others I have seen. I
Generally one should not dispatch on:
 - `DenseArray` it is **not** the complement of sparse array. There are lots of subtypes of `AbstractArray`, most of which are not obviously sparse, nor are they subtypes of `DenseArray`. In particular wrapper array types that can wrap dense or sparse do not subtype it, e.g. `Symmetric`
 - `AbstractFloat`, can almost always be relaxed to `Real`
 - `DataType`: this will excude the type of `Union`s and `UnionAll`s, so use `Type` instead, unless that is the goal.
